In [1]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='07-sklearn.ensemble.b-boosting'
classifierName='AdaBoostClassifier'
extraParameterName='4-Random oversampling'

print('done')

done


In [2]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

done


In [3]:
# Results

estimator__max_depthFound=7
# learning too low makes the process very slow
learning_rateFound=0.1
n_estimatorsFound=54

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,estimator__max_depthFound,n_estimatorsFound)


print('done')

done


In [4]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

,db_uuid,trx_date_time,term_mcc,term_country,mcd_fraud_score,vaa_score,TRX_3D_SECURED,trx_accepted,trx_cnp,card_pan_id,...,clusterCardHolder_cluster4,clusterMerchant_M0,clusterMerchant_M1,clusterMerchant_M2,clusterMerchant_M3,clusterMerchant_UNKNOWN,nbPreviousTrx,previousTrxAmountSumLog,trx_amount_log10,distancePrevTrx
0,A3DEF420A53B11EFABA49D2778D26ADA,2024-11-18 00:25:43,5499,NLD,289.000000,26.258093,0,1,1,A4BF631BFD796F0D7D9A714E5AD1F5D26E8D70A4999F30...,...,0,1,0,0,0,0,0,0.0,1.763353,1.100000
1,31BFAC76A53C11EFABA49D2778D26ADA,2024-11-18 00:30:44,4215,BEL,148.000000,26.258093,1,1,1,6048F88F23C4D90D50615342B2445A0CD680E3D102E56C...,...,0,0,0,0,1,0,0,0.0,1.330617,0.768071
2,75A6059CA53811EFA6F9EF55A8E51A55,2024-11-18 00:03:02,5968,FRA,363.000000,26.258093,0,1,1,6081849BC94C237C841CDC43E06CAB46A25F9BE5DE3DF3...,...,0,1,0,0,0,0,0,0.0,0.902547,1.100000
3,C5DEFDAAA53A11EFAAA0DF784AB5695E,2024-11-18 00:09:37,5812,USA,225.222235,26.258093,0,1,0,60E1D8C417CDFDE242B10454048AEEF15CF15C39251BE1...,...,0,0,0,0,0,1,0,0.0,1.563837,1.000000
4,D4F9F3CCA53E11EFAB1F43D36D9BDFD5,2024-11-18 00:48:47,5942,LUX,225.222235,30.000000,0,1,0,AFEEC568C21C0640F97EACED2F4D9B25537D94CE391457...,...,1,1,0,0,0,0,0,0.0,1.546049,1.100000


In [5]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.randomOverSampling(x_train0, y_train0)

Before Sampling shape (847879, 39) fraud rate 0.00123 
Duration 0.9 s 
After Sampling shape  (852039, 39) fraud rate 0.00614 


# Hyperparameters tuning


In [7]:
%%script false

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

modelClf = AdaBoostClassifier(random_state=42)
dic_param={
    'estimator': [DecisionTreeClassifier()],
    'estimator__max_depth':randint(1,9),
    'n_estimators': randint(35,90),
    'learning_rate': [0.1]
}

res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train,iter=4)


print(res)

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 7, 'learning_rate': 0.1, 'n_estimators': 54}
#0.7725600175101597
#score   0.8136319794234272


Couldn't find program: 'false'


In [8]:
#%%script false
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

modelClf = AdaBoostClassifier(random_state=42)

 
dic_param={
     'estimator': [DecisionTreeClassifier()],
    'estimator__max_depth':[6,7,8],
    'n_estimators':[50,54,55],
    'learning_rate':[0.1]
}

res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)





Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START estimator=DecisionTreeClassifier(), estimator__max_depth=6, learning_rate=0.1, n_estimators=50
[CV 1/5; 1/9] END estimator=DecisionTreeClassifier(), estimator__max_depth=6, learning_rate=0.1, n_estimators=50;, score=0.632 total time= 7.1min
[CV 2/5; 1/9] START estimator=DecisionTreeClassifier(), estimator__max_depth=6, learning_rate=0.1, n_estimators=50
[CV 2/5; 1/9] END estimator=DecisionTreeClassifier(), estimator__max_depth=6, learning_rate=0.1, n_estimators=50;, score=0.641 total time= 7.2min
[CV 3/5; 1/9] START estimator=DecisionTreeClassifier(), estimator__max_depth=6, learning_rate=0.1, n_estimators=50
[CV 3/5; 1/9] END estimator=DecisionTreeClassifier(), estimator__max_depth=6, learning_rate=0.1, n_estimators=50;, score=0.598 total time= 7.2min
[CV 4/5; 1/9] START estimator=DecisionTreeClassifier(), estimator__max_depth=6, learning_rate=0.1, n_estimators=50
[CV 4/5; 1/9] END estimator=DecisionTreeCl

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()



modelClf = AdaBoostClassifier(random_state=42)
parameters={'learning_rate': learning_rateFound, 'n_estimators':n_estimatorsFound, 'estimator':DecisionTreeClassifier(),
           'estimator__max_depth':estimator__max_depthFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')


fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD07C-AdaBoostClassifierProbaHistogram' if saveImage else None)



In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

